In [1]:
import geojson
import pandas as pd
import shapely.wkt
from shapely import geometry
import descarteslabs as dl
import pickle as pkl
import geopandas as gpd
from random import sample
from tqdm.notebook import tqdm
import csv
import math
import numpy as np

In [2]:
datasetName='happy_cow'
country='nepal'
stateSeparatedFile='updatedModel_state_sampled_nepal'
datasetFile='updatedModel_state_sampled_happy_cow'
maxKilnStateSample=500
maxNonKilnStateSample=250
minNonKilnStateSample=50
randomSample=500
oversampleStates=['F.A.T.A','N.W.F.P','Punjab']
oversampleModifier=1.5

In [21]:
#Load Datasets
highDataset={'tile_key':{},'geometry':{},'prediction':{},'area':{}}
restDataset={'tile_key':{},'geometry':{},'prediction':{},'area':{}}
totalDataset=pd.read_csv('../Predicted_Datasets/'+datasetName+'_results.csv').to_dict()
highcount=0
lowcount=0
for idx in tqdm(totalDataset['prediction']):
    if totalDataset['prediction'][idx] >= .99:
        highDataset['tile_key'][highcount]=totalDataset['tile_key'][idx]
        highDataset['geometry'][highcount]=totalDataset['geometry'][idx]
        highDataset['prediction'][highcount]=totalDataset['prediction'][idx]
        highcount+=1
    else:
        restDataset['tile_key'][lowcount]=totalDataset['tile_key'][idx]
        restDataset['geometry'][lowcount]=totalDataset['geometry'][idx]
        restDataset['prediction'][lowcount]=totalDataset['prediction'][idx]
        lowcount+=1
print(highcount)
with open('../GeoJSONS/Brick_Beltv2.geojson') as f:
    brick_belt = geojson.load(f) 
with open('../GeoJSONS/Bangladesh_States.geojson') as f:
    states=geojson.load(f)

  0%|          | 0/296960 [00:00<?, ?it/s]

163


In [22]:
#Create shapely brick belt
shapelyBelt=geometry.shape(brick_belt['features'][0]['geometry'])

In [23]:
#Prepare to make a geodataframe from the geometry of the indian states, also take the time to calculate non-belt max area
shapelyStates={}
togpd=[]
names=[]
within={}
max_area=0
for x in states['features']:
    if shapelyBelt.intersects(geometry.shape(x['geometry'])):
        within = shapelyBelt.intersection(geometry.shape(x['geometry'])).area/geometry.shape(x['geometry']).area >= .25
    else:
        within=False
    shapelyStates[x['properties']['NAME_1']]={
        'geometry':geometry.shape(x['geometry']),
        'within':within,
        'tiles':[]
    }
    if not shapelyStates[x['properties']['NAME_1']]['within']:
        if max_area<shapelyStates[x['properties']['NAME_1']]['geometry'].area:
            max_area=shapelyStates[x['properties']['NAME_1']]['geometry'].area
    togpd.append(geometry.shape(x['geometry']))
    names.append(x['properties']['NAME_1'])

In [27]:
print(len(togpd))

8


In [28]:
#Create state geodataframe
gdf = gpd.GeoDataFrame(data={'name':names}, crs='epsg:4326', geometry=togpd)

## UNCOMENT THE TWO BOXES BELOW IF YOUR TILE DATA NEEDS TO BE REASSESED OR YOU NEED TO MAKE A NEW POINT-STATE GEOMETRY
## (DON'T FORGET TO CHANGE ANS FILE NAME)

In [29]:
#"""
#Iterate through all of our keys and obtain the geometric center and the respective tile_keys
centers=[]
tile_keys=[]
for x in tqdm(highDataset['tile_key'].keys()):
    centers.append(dl.scenes.DLTile.from_key(highDataset['tile_key'][x]).geometry.centroid)
    tile_keys.append(highDataset['tile_key'][x])
    
#Create the tile geodataframe from the center points 
gdfp = gpd.GeoDataFrame(data={'tile_key':tile_keys,'idx':range(len(tile_keys))},crs='epsg:4326', geometry=centers)
#"""

  0%|          | 0/163 [00:00<?, ?it/s]

In [30]:
print(centers[0])

POINT (81.07496437662 28.15667017686067)


In [8]:
#"""
#Create and save the joined point-state geometry as a csv
ans = gpd.tools.sjoin(gdfp, gdf, predicate="within", how='left')
ans.drop('geometry',axis=1).to_csv('State_Separated_Datasets/'+stateSeparatedFile+'.csv')
#"""

In [9]:
#load ans
ans=pd.read_csv('State_Separated_Datasets/'+stateSeparatedFile+'.csv').to_dict()

In [10]:
#Iterate through each tile once more and add each tile to it's respective state list
stateTiles={}
for x in shapelyStates.keys():
    stateTiles[x]=[]
for x in tqdm(range(len(ans['tile_key']))):
    if type(ans['name'][x])==str:
        stateTiles[ans['name'][x]].append([ans['tile_key'][x],ans['idx'][x]])

  0%|          | 0/163 [00:00<?, ?it/s]

In [11]:
with open('../Predicted_Datasets/'+datasetFile+'_results.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows([['tile_key','geometry','prediction','area']])
    
#Iterate through each state and sample respective amounts per state, then save it to the sampled csv.    
for x in shapelyStates.keys():
    print('Working on '+x+'!')
    if x in oversampleStates:
        print('Oversampling!')
        modifier=oversampleModifier
    else:
        modifier=1
    if shapelyStates[x]['within']:
        print('Inside Brick Belt!')
        samplesize=min([len(stateTiles[x]),round(maxKilnStateSample*modifier)])
    else:
        print('Outside Brick Belt!')
        cur_area=shapelyStates[x]['geometry'].area
        samplesize=min([round(maxNonKilnStateSample*cur_area/max_area*modifier),len(stateTiles[x])])
        if round(minNonKilnStateSample*modifier) > len(stateTiles[x]):
            samplesize=len(stateTiles[x])
        else:
            samplesize=max([round(minNonKilnStateSample*modifier),samplesize])
    print('Sampling '+str(samplesize)+' tiles!')
    shapelyStates[x]['tiles']=sample(stateTiles[x],samplesize)
    rows=[]
    print('Saving!')
    for y in range(len(shapelyStates[x]['tiles'])):
        rows.append([highDataset['tile_key'][shapelyStates[x]['tiles'][y][1]],highDataset['geometry'][shapelyStates[x]['tiles'][y][1]],highDataset['prediction'][shapelyStates[x]['tiles'][y][1]],x])
    with open('../Predicted_Datasets/'+datasetFile+'_results.csv', 'a', newline='') as file:
        writer = csv.writer(file)
        writer.writerows(rows)
if randomSample:
    print('Sampling randomly!')
    sampledkeys=sample(list(restDataset['tile_key'].keys()),min([len(restDataset['tile_key'].keys()),randomSample]))
    print('Sampling',len(sampledkeys),'tiles!')
    rows=[]
    print('Saving!')
    for x in sampledkeys:
        rows.append([restDataset['tile_key'][x],restDataset['geometry'][x],restDataset['prediction'][x],'random'])
    with open('../Predicted_Datasets/'+datasetFile+'_results.csv', 'a', newline='') as file:
        writer = csv.writer(file)
        writer.writerows(rows)

Working on Province No. 1!
Inside Brick Belt!
Sampling 27 tiles!
Saving!
Working on Province No. 2!
Inside Brick Belt!
Sampling 53 tiles!
Saving!
Working on Bagmati Pradesh!
Inside Brick Belt!
Sampling 6 tiles!
Saving!
Working on Gandaki Pradesh!
Inside Brick Belt!
Sampling 5 tiles!
Saving!
Working on Province No. 5!
Inside Brick Belt!
Sampling 13 tiles!
Saving!
Working on Karnali Pradesh!
Inside Brick Belt!
Sampling 0 tiles!
Saving!
Working on Sudurpashchim Pradesh!
Inside Brick Belt!
Sampling 0 tiles!
Saving!
Sampling randomly!
Sampling 500 tiles!
Saving!


In [12]:
sampledDataset=pd.read_csv('../Predicted_Datasets/'+datasetFile+'_results.csv').to_dict()

In [13]:
import descarteslabs.workflows as wf
from ipyleaflet import GeoJSON
m = wf.interactive.MapApp()
for x in tqdm(sampledDataset['area'].keys()):
    #if sampledDataset['area'][x]=='Andhra Pradesh':
        subtilegjs=GeoJSON(
           data=geojson.FeatureCollection([dl.scenes.DLTile.from_key(sampledDataset['tile_key'][x]).__geo_interface__]),
           style={"color":"red",
                  "fillOpacity":0}
        )
        m.add_layer(subtilegjs)


/opt/conda/lib/python3.7/site-packages/descarteslabs/_dl_modules/common/workflows/arrow_serialization/context.py:57: FutureWarning: 'pyarrow.SerializationContext' is deprecated and will be removed in a future version. Use pickle or the pyarrow IPC functionality instead.
  serialization_context = pa.SerializationContext()
/opt/conda/lib/python3.7/site-packages/descarteslabs/_dl_modules/common/workflows/arrow_serialization/context.py:58: FutureWarning: 'pyarrow.register_default_serialization_handlers' is deprecated as of 2.0.0 and will be removed in a future version. Use pickle or the pyarrow IPC functionality instead.
  pa.register_default_serialization_handlers(serialization_context)


  0%|          | 0/604 [00:00<?, ?it/s]

In [14]:
m


`ipyleaflet` and/or `ipywidgets` Jupyter extensions are not installed! (or you're not in a Jupyter notebook.)
To install for JupyterLab, run this in a cell:
    !jupyter labextension install jupyter-leaflet @jupyter-widgets/jupyterlab-manager
To install for plain Jupyter Notebook, run this in a cell:
    !jupyter nbextension enable --py --sys-prefix ipyleaflet
Then, restart the kernel and refresh the webpage.


In [15]:
print('hi')

hi
